In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os, zipfile, shutil, tensorflow as tf
from tensorflow.keras import mixed_precision


mixed_precision.set_global_policy('mixed_float16')
tf.config.optimizer.set_jit(True)


zip_path = '/content/drive/MyDrive/archive.zip'
extract_dir = '/content/asl_alphabet'

if not os.path.exists(extract_dir):
    print("Extracting archive.zip...")
    shutil.copy(zip_path, '/content/archive.zip')
    try:
        with zipfile.ZipFile('/content/archive.zip', 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print("Extraction successful.")
    except zipfile.BadZipFile:
        print("Error: Not a valid zip file.")
else:
    print("Dataset already extracted.")

train_dir = os.path.join(extract_dir, 'asl_alphabet_train', 'asl_alphabet_train')
image_size = (100, 100)
batch_size = 512

def preprocess(image, label):
    image = tf.image.resize(image, image_size)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, 0.1)
    image = tf.image.random_contrast(image, 0.9, 1.1)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

AUTOTUNE = tf.data.AUTOTUNE


train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset='training',
    seed=42,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset='validation',
    seed=42,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical'
)


train_ds = (train_ds
            .map(preprocess, num_parallel_calls=AUTOTUNE)
            .shuffle(buffer_size=5000)
            .cache()
            .prefetch(AUTOTUNE))

val_ds = (val_ds
          .map(preprocess, num_parallel_calls=AUTOTUNE)
          .cache()
          .prefetch(AUTOTUNE))


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(29, activation='softmax', dtype='float32')
])

model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True, monitor='val_loss', verbose=1)


history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[early_stop, checkpoint]
)


Mounted at /content/drive
Extracting archive.zip...
Extraction successful.
Found 87000 files belonging to 29 classes.
Using 69600 files for training.
Found 87000 files belonging to 29 classes.
Using 17400 files for validation.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 98, 98, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 49, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 47, 47, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 33856)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,333,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 29)             │         3,741 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,356,829 (16.62 MB)

...

136/136 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.7280 - loss: 0.8236 - val_accuracy: 0.8672 - val_loss: 0.5192
Epoch 7/10
136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7582 - loss: 0.7225
Epoch 7: val_loss improved from 0.51916 to 0.41922, saving model to best_model.h5


136/136 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.7582 - loss: 0.7223 - val_accuracy: 0.8988 - val_loss: 0.4192
Epoch 8/10
136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7885 - loss: 0.6299
Epoch 8: val_loss improved from 0.41922 to 0.35459, saving model to best_model.h5


136/136 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.7885 - loss: 0.6298 - val_accuracy: 0.9134 - val_loss: 0.3546
Epoch 9/10
136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8067 - loss: 0.5657
Epoch 9: val_loss improved from 0.35459 to 0.32802, saving model to best_model.h5


136/136 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.8067 - loss: 0.5657 - val_accuracy: 0.9172 - val_loss: 0.3280
Epoch 10/10
136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8268 - loss: 0.5140
Epoch 10: val_loss improved from 0.32802 to 0.27975, saving model to best_model.h5


136/136 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.8268 - loss: 0.5140 - val_accuracy: 0.9332 - val_loss: 0.2797


In [2]:

drive_path = '/content/drive/MyDrive/asl_model_saves'
os.makedirs(drive_path, exist_ok=True)


model.save(f"{drive_path}/best_model.keras")

print(" Model saved to:", f"{drive_path}/best_model.keras")


 Model saved to: /content/drive/MyDrive/asl_model_saves/best_model.keras
